In [12]:
import pygame
import random
import sys

def get_score_probability(kicker_side, goalie_side):
    """
    Returns the kicker's probability of scoring.
    If kicker_side == goalie_side => lower probability (0.3).
    If opposite sides => higher probability (0.8).
    """
    if kicker_side == goalie_side:
        return 0.3
    else:
        return 0.8

def main():
    pygame.init()
    WIDTH, HEIGHT = 800, 600
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Penalty Kick Game (Slider + Bug Fix)")

    # Colors
    GREEN = (34, 139, 34)
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    RED   = (220, 20, 60)
    BLUE  = (70, 130, 180)
    DARK_GRAY = (50, 50, 50)
    LIGHT_GRAY = (170, 170, 170)
    GRAY  = (100, 100, 100)

    # Fonts
    font_small  = pygame.font.SysFont(None, 24)
    font_medium = pygame.font.SysFont(None, 32)
    font_large  = pygame.font.SysFont(None, 48)

    # Top bar height
    top_bar_height = 160

    # Probability that goalie dives left
    p = 0.5

    # Total attempts
    total_attempts = 5

    # Tracking variables
    attempt_count = 1
    goals_scored = 0
    shots_left = 0
    shots_right = 0
    goalie_dives_left_count = 0
    goalie_dives_right_count = 0

    # Field geometry
    goal_line_y = top_bar_height + 40
    goalie_w, goalie_h = 60, 60
    goalie_x = (WIDTH - goalie_w) // 2
    goalie_y = goal_line_y - goalie_h // 2

    ball_radius = 20
    ball_x = WIDTH // 2
    ball_y = HEIGHT - 100

    # State machine
    # WAIT_FOR_INPUT -> SHOW_RESULT -> GAME_OVER
    state = "WAIT_FOR_INPUT"
    result_text = ""
    result_time = 0
    result_duration = 2000  # ms

    clock = pygame.time.Clock()

    # Buttons for "Kick LEFT" / "Kick RIGHT" at the bottom
    btn_left_rect  = pygame.Rect(50, HEIGHT - 80, 150, 50)
    btn_right_rect = pygame.Rect(WIDTH - 200, HEIGHT - 80, 150, 50)

    # We’ll keep a "Restart" button in the top bar
    btn_restart_rect = pygame.Rect(WIDTH - 140, 110, 120, 40)

    # Slider geometry (for adjusting p)
    slider_x = 520
    slider_y = 80
    slider_width = 200
    slider_height = 5
    handle_radius = 10

    # We'll track if we're dragging the slider
    dragging_slider = False

    def slider_rect():
        # The region around the slider bar that we consider clickable
        return pygame.Rect(slider_x, slider_y - handle_radius,
                           slider_width, handle_radius * 2)

    def reset_game():
        nonlocal attempt_count, goals_scored
        nonlocal shots_left, shots_right
        nonlocal goalie_dives_left_count, goalie_dives_right_count
        nonlocal state, result_text, result_time

        attempt_count = 1
        goals_scored = 0
        shots_left = 0
        shots_right = 0
        goalie_dives_left_count = 0
        goalie_dives_right_count = 0

        state = "WAIT_FOR_INPUT"
        result_text = ""
        result_time = 0

    running = True
    while running:
        dt = clock.tick(60)  # up to 60 FPS

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            # MOUSE DOWN
            if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                mx, my = pygame.mouse.get_pos()

                # Check if clicked "Restart"
                if btn_restart_rect.collidepoint(mx, my):
                    reset_game()

                # Check if clicked on the slider handle or bar
                if slider_rect().collidepoint(mx, my):
                    dragging_slider = True
                    # Update p based on mouse x
                    p = (mx - slider_x) / slider_width
                    p = max(0.0, min(1.0, p))

                # If we are in WAIT_FOR_INPUT, check if user clicked "Kick Left" or "Kick Right"
                if state == "WAIT_FOR_INPUT":
                    if btn_left_rect.collidepoint(mx, my):
                        # Kicker chooses Left
                        shots_left += 1
                        kicker_side = 0  # 0 for left

                        # Goalie random choice
                        goalie_side = 0 if (random.random() < p) else 1
                        if goalie_side == 0:
                            goalie_dives_left_count += 1
                        else:
                            goalie_dives_right_count += 1

                        # Probability of scoring
                        score_prob = get_score_probability(kicker_side, goalie_side)
                        shot_success = (random.random() < score_prob)
                        result_text = "GOAL!" if shot_success else "MISS!"
                        if shot_success:
                            goals_scored += 1

                        state = "SHOW_RESULT"
                        result_time = 0

                    elif btn_right_rect.collidepoint(mx, my):
                        # Kicker chooses Right
                        shots_right += 1
                        kicker_side = 1  # 1 for right

                        # Goalie random choice
                        goalie_side = 0 if (random.random() < p) else 1
                        if goalie_side == 0:
                            goalie_dives_left_count += 1
                        else:
                            goalie_dives_right_count += 1

                        # Probability of scoring
                        score_prob = get_score_probability(kicker_side, goalie_side)
                        shot_success = (random.random() < score_prob)
                        result_text = "GOAL!" if shot_success else "MISS!"
                        if shot_success:
                            goals_scored += 1

                        state = "SHOW_RESULT"
                        result_time = 0

            # MOUSE UP
            if event.type == pygame.MOUSEBUTTONUP and event.button == 1:
                dragging_slider = False

            # MOUSE MOTION (for dragging the slider)
            if event.type == pygame.MOUSEMOTION:
                if dragging_slider:
                    mx, my = event.pos
                    # Update p based on horizontal position
                    p = (mx - slider_x) / slider_width
                    p = max(0.0, min(1.0, p))

            # If in GAME_OVER, allow ESC to quit
            if state == "GAME_OVER":
                if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                    running = False

        # Update logic
        if state == "SHOW_RESULT":
            result_time += dt
            if result_time >= result_duration:
                # Only increment attempt if we haven't reached total_attempts
                if attempt_count < total_attempts:
                    attempt_count += 1
                    state = "WAIT_FOR_INPUT"
                else:
                    state = "GAME_OVER"

        # --- DRAW ---
        screen.fill(GREEN)

        # Draw a top bar
        top_bar_rect = pygame.Rect(0, 0, WIDTH, top_bar_height)
        pygame.draw.rect(screen, DARK_GRAY, top_bar_rect)

        # Field line
        pygame.draw.line(screen, WHITE, (0, goal_line_y), (WIDTH, goal_line_y), 4)

        # Goalie
        pygame.draw.rect(screen, BLUE, (goalie_x, goalie_y, goalie_w, goalie_h))

        # Ball
        pygame.draw.circle(screen, WHITE, (ball_x, ball_y), ball_radius)

        # --- STATS ---
        # Attempt, Goals, Rate
        # We'll display attempt_count, but clamp visually if it's > total_attempts
        displayed_attempt = min(attempt_count, total_attempts)
        shots_completed = max(0, displayed_attempt - 1)  # how many shots are fully done
        success_rate = (goals_scored / shots_completed * 100) if shots_completed > 0 else 0.0

        stats_line1 = f"Attempt: {displayed_attempt}/{total_attempts}   Goals: {goals_scored}   Rate: {success_rate:.1f}%"
        txt1_surf = font_medium.render(stats_line1, True, WHITE)
        screen.blit(txt1_surf, (20, 10))

        # Shots & Goalie stats
        stats_line2 = (f"Shots(L)={shots_left},(R)={shots_right} | "
                       f"Goalie(L)={goalie_dives_left_count},(R)={goalie_dives_right_count}")
        txt2_surf = font_small.render(stats_line2, True, WHITE)
        screen.blit(txt2_surf, (20, 50))

        # Probability label
        prob_label_surf = font_small.render("Goalie Dive Probability", True, WHITE)
        prob_label_rect = prob_label_surf.get_rect(center=(slider_x + slider_width/2, 30))
        screen.blit(prob_label_surf, prob_label_rect)

        # Draw the slider bar
        pygame.draw.rect(screen, GRAY, (slider_x, slider_y, slider_width, slider_height))

        # Draw the slider handle
        handle_x = slider_x + int(p * slider_width)
        handle_y = slider_y + slider_height // 2
        pygame.draw.circle(screen, LIGHT_GRAY, (handle_x, handle_y), handle_radius)

        # Show the numeric value of p
        p_value_surf = font_small.render(f"{p:.2f}", True, WHITE)
        p_value_rect = p_value_surf.get_rect(center=(slider_x + slider_width/2, slider_y + 35))
        screen.blit(p_value_surf, p_value_rect)

        # Restart button
        pygame.draw.rect(screen, LIGHT_GRAY, btn_restart_rect)
        restart_txt = font_medium.render("Restart", True, BLACK)
        restart_rect = restart_txt.get_rect(center=btn_restart_rect.center)
        screen.blit(restart_txt, restart_rect)

        # If in SHOW_RESULT, display GOAL or MISS
        if state == "SHOW_RESULT":
            color = RED if "MISS" in result_text else WHITE
            result_surf = font_large.render(result_text, True, color)
            result_rect = result_surf.get_rect(center=(WIDTH // 2, HEIGHT // 2))
            screen.blit(result_surf, result_rect)

        # If GAME_OVER, show final message
        if state == "GAME_OVER":
            final_rate = (goals_scored / float(total_attempts)) * 100 if total_attempts else 0
            game_over_text = f"Game Over! You scored {goals_scored}/{total_attempts}  ({final_rate:.1f}%)."
            go_surf = font_large.render(game_over_text, True, BLACK)
            go_rect = go_surf.get_rect(center=(WIDTH // 2, HEIGHT // 2))
            screen.blit(go_surf, go_rect)

            exit_text = "Press ESC or close window to exit."
            exit_surf = font_medium.render(exit_text, True, BLACK)
            exit_rect = exit_surf.get_rect(center=(WIDTH // 2, HEIGHT // 2 + 50))
            screen.blit(exit_surf, exit_rect)

        # --- BOTTOM BUTTONS ---
        pygame.draw.rect(screen, LIGHT_GRAY, btn_left_rect)
        left_txt = font_medium.render("Kick LEFT", True, BLACK)
        left_txt_rect = left_txt.get_rect(center=btn_left_rect.center)
        screen.blit(left_txt, left_txt_rect)

        pygame.draw.rect(screen, LIGHT_GRAY, btn_right_rect)
        right_txt = font_medium.render("Kick RIGHT", True, BLACK)
        right_txt_rect = right_txt.get_rect(center=btn_right_rect.center)
        screen.blit(right_txt, right_txt_rect)

        pygame.display.flip()

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 